In [20]:
import sys,subprocess
pkgs=["autogluon.tabular==1.4.0","catboost==1.2.8","tqdm"]
subprocess.check_call([sys.executable,"-m","pip","install","-q"]+pkgs)


0

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from autogluon.features.generators import AutoMLPipelineFeatureGenerator
from catboost import CatBoostRegressor,Pool
from catboost.utils import get_gpu_device_count
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tqdm.auto import tqdm

from pathlib import Path
import pandas as pd

path = Path(".")





In [3]:
train = pd.read_csv(path/"train_home.csv")
test = pd.read_csv(path/"test_home.csv")

In [4]:
train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [5]:
test

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [6]:
target_col = "SalePrice"  # или как называется таргет в новом датасете
feature_cols = [c for c in train.columns if c != target_col and c in test.columns]

X = train[feature_cols]
y = train[target_col].values
X_test = test[feature_cols]

In [9]:
print("До автофичей:", X.shape)          # (41105, N_исходных_фич)
print("Число исходных фич:", X.shape[1])


До автофичей: (1460, 80)
Число исходных фич: 80


In [8]:
from autogluon.features.generators import AutoMLPipelineFeatureGenerator

fg = AutoMLPipelineFeatureGenerator(
    enable_numeric_features=True,
    enable_categorical_features=True,
    enable_datetime_features=True,
    enable_text_special_features=True,
    enable_text_ngram_features=True,
    enable_raw_text_features=False,
)

X_tr_feat = fg.fit_transform(X=X, y=y)
X_te_feat = fg.transform(X_test)

print(X_tr_feat.shape, X_te_feat.shape)
print(X_tr_feat.columns[:30])



(1460, 80) (1459, 80)
Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'Street', 'Utilities',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'CentralAir',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars'],
      dtype='object')


In [10]:
print("После автофичей:", X_tr_feat.shape)   # (41105, 916)
print("Число фич после генерации:", X_tr_feat.shape[1])


После автофичей: (1460, 80)
Число фич после генерации: 80


In [11]:
from catboost.utils import get_gpu_device_count
import catboost, sys
print("python:", sys.executable)
print("catboost version:", catboost.__version__)
print("GPU devices seen by catboost:", get_gpu_device_count())


python: /venv/main/bin/python
catboost version: 1.2.8
GPU devices seen by catboost: 1


In [12]:
import numpy as np

cat_cols = [
    c for c in X_tr_feat.columns
    if str(X_tr_feat[c].dtype) in ("category", "object")
]

for c in cat_cols:
    X_tr_feat[c] = X_tr_feat[c].astype(str).replace({"nan": "missing"}).fillna("missing")
    if c in X_te_feat.columns:
        X_te_feat[c] = X_te_feat[c].astype(str).replace({"nan": "missing"}).fillna("missing")

print("Категориальных колонок:", len(cat_cols))


Категориальных колонок: 40


In [13]:
cat_features = [
    c for c in X_tr_feat.columns
    if str(X_tr_feat[c].dtype) in ("object",)  # теперь всё строковое
]
print(len(cat_features), "cat_features")


40 cat_features


In [14]:
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error
import numpy as np

idx = np.arange(len(X_tr_feat))
np.random.seed(42)
np.random.shuffle(idx)
small_idx = idx[:1000]

X_small = X_tr_feat.iloc[small_idx]
y_small = y[small_idx]

train_pool = Pool(X_small, y_small, cat_features=cat_features)

model_gpu_test = CatBoostRegressor(
    loss_function="RMSE",
    eval_metric="RMSE",
    iterations=300,
    learning_rate=0.05,
    depth=6,
    task_type="GPU",
    devices="0",
    verbose=50,
    random_seed=42,
)

print("=== GPU sanity fit (300 iters on 1000 rows) ===")
model_gpu_test.fit(train_pool)


=== GPU sanity fit (300 iters on 1000 rows) ===
0:	learn: 79822.0184128	total: 23.3ms	remaining: 6.97s
50:	learn: 28951.1496753	total: 974ms	remaining: 4.76s
100:	learn: 21136.8504901	total: 1.94s	remaining: 3.81s
150:	learn: 18114.3798295	total: 2.87s	remaining: 2.84s
200:	learn: 15758.4885529	total: 3.83s	remaining: 1.89s
250:	learn: 14344.2256402	total: 4.79s	remaining: 935ms
299:	learn: 13164.8370247	total: 5.62s	remaining: 0us


In [15]:
import pandas as pd
import numpy as np

for c in X_tr_feat.columns:
    if X_tr_feat[c].dtype == "object":
        X_tr_feat[c] = X_tr_feat[c].fillna("missing").astype(str)
        if c in X_te_feat.columns:
            X_te_feat[c] = X_te_feat[c].fillna("missing").astype(str)
    else:
        X_tr_feat[c] = pd.to_numeric(X_tr_feat[c], errors="coerce")
        if c in X_te_feat.columns:
            X_te_feat[c] = pd.to_numeric(X_te_feat[c], errors="coerce")

cat_features = [
    i for i, c in enumerate(X_tr_feat.columns)
    if X_tr_feat[c].dtype == "object"
]

print("cat_features count:", len(cat_features))

cat_features count: 40


In [ ]:
'''надо поменять вообще было функцию на MAE для этой соревы но я проебался малек'''

In [16]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor, Pool
from tqdm.auto import tqdm
import numpy as np

def rmse_func(y_true, y_pred):
    return mean_squared_error(y_true, y_pred) ** 0.5

N_SPLITS = 5
N_ITER = 3000
DEPTH = 8
OD_WAIT = 300
TASK_TYPE = "GPU"

X = X_tr_feat
y_arr = y

kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

oof = np.zeros(len(X))
test_pred = np.zeros(len(X_te_feat))
scores = []

for fold, (tr_idx, val_idx) in enumerate(tqdm(kf.split(X), total=N_SPLITS, desc="Folds"), 1):
    X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
    y_tr, y_val = y_arr[tr_idx], y_arr[val_idx]

    train_pool = Pool(X_tr, y_tr, cat_features=cat_features)
    val_pool = Pool(X_val, y_val, cat_features=cat_features)

    params = dict(
        loss_function="RMSE",
        eval_metric="RMSE",
        iterations=N_ITER,
        learning_rate=0.01,
        depth=DEPTH,
        l2_leaf_reg=3.0,
        random_seed=42 + fold,
        verbose=200,
        od_type="Iter",
        od_wait=OD_WAIT,
        task_type=TASK_TYPE,
    )

    if TASK_TYPE == "GPU":
        params["devices"] = "0"

    print(f"\n=== Fold {fold}/{N_SPLITS} ===")
    model = CatBoostRegressor(**params)
    model.fit(train_pool, eval_set=val_pool, use_best_model=True)

    val_pred = model.predict(X_val)
    oof[val_idx] = val_pred
    rmse = rmse_func(y_val, val_pred)
    scores.append(rmse)
    print(f"Fold {fold} RMSE: {rmse:.5f}")

    bs = 1024
    fold_test = np.zeros(len(X_te_feat))
    for s in tqdm(range(0, len(X_te_feat), bs), leave=False, desc=f"Predict fold {fold}"):
        e = s + bs
        fold_test[s:e] = model.predict(X_te_feat.iloc[s:e])
    test_pred += fold_test / N_SPLITS

oof_rmse = rmse_func(y_arr, oof)
print("\nfold_rmse:", scores)
print("mean_rmse:", np.mean(scores))
print("oof_rmse:", oof_rmse)

Folds:   0%|          | 0/5 [00:00<?, ?it/s]


=== Fold 1/5 ===
0:	learn: 76701.9648263	test: 87085.5667994	best: 87085.5667994 (0)	total: 39ms	remaining: 1m 56s
200:	learn: 31491.7596597	test: 40329.7623320	best: 40329.7623320 (200)	total: 8.12s	remaining: 1m 53s
400:	learn: 23528.1093838	test: 32419.3022159	best: 32419.3022159 (400)	total: 15.8s	remaining: 1m 42s
600:	learn: 20970.3082045	test: 29958.4285339	best: 29958.4285339 (600)	total: 23.6s	remaining: 1m 34s
800:	learn: 19498.7820905	test: 29001.2253005	best: 29001.2253005 (800)	total: 31.3s	remaining: 1m 25s
1000:	learn: 18502.6537778	test: 28404.3515555	best: 28404.3515555 (1000)	total: 39.2s	remaining: 1m 18s
1200:	learn: 17559.5554223	test: 28017.6967325	best: 28017.6967325 (1200)	total: 47.4s	remaining: 1m 11s
1400:	learn: 16899.2906717	test: 27709.9012777	best: 27709.9012777 (1400)	total: 55.8s	remaining: 1m 3s
1600:	learn: 16337.1865454	test: 27411.6861790	best: 27411.6861790 (1600)	total: 1m 3s	remaining: 55.9s
1800:	learn: 15928.4105821	test: 27216.8864263	best: 2

Predict fold 1:   0%|          | 0/2 [00:00<?, ?it/s]


=== Fold 2/5 ===
0:	learn: 78044.8891189	test: 82139.2900550	best: 82139.2900550 (0)	total: 43.3ms	remaining: 2m 9s
200:	learn: 31837.1201561	test: 35505.6845890	best: 35505.6845890 (200)	total: 8.3s	remaining: 1m 55s
400:	learn: 23445.5581610	test: 29247.8302204	best: 29247.8302204 (400)	total: 16.5s	remaining: 1m 46s
600:	learn: 20722.8546190	test: 27707.9461744	best: 27707.1553866 (599)	total: 24.5s	remaining: 1m 37s
800:	learn: 19116.4721913	test: 26978.0567636	best: 26972.7693348 (799)	total: 32.5s	remaining: 1m 29s
1000:	learn: 18163.1828241	test: 26582.2570550	best: 26582.2570550 (1000)	total: 40.5s	remaining: 1m 20s
1200:	learn: 17611.3290561	test: 26414.1901590	best: 26412.5395873 (1177)	total: 48.7s	remaining: 1m 12s
1400:	learn: 17230.7771790	test: 26229.9394011	best: 26225.1986320 (1386)	total: 56.7s	remaining: 1m 4s
1600:	learn: 16908.9307099	test: 26107.6461586	best: 26107.6461586 (1600)	total: 1m 4s	remaining: 56.5s
1800:	learn: 16560.9502738	test: 25959.9797201	best: 2

Predict fold 2:   0%|          | 0/2 [00:00<?, ?it/s]

...


=== Fold 4/5 ===
0:	learn: 78915.8704514	test: 78754.7343795	best: 78754.7343795 (0)	total: 39.8ms	remaining: 1m 59s
200:	learn: 31300.1009163	test: 36517.9711060	best: 36517.9711060 (200)	total: 8.22s	remaining: 1m 54s
400:	learn: 22641.8654152	test: 31130.6381226	best: 31130.6381226 (400)	total: 16.5s	remaining: 1m 46s
600:	learn: 19873.2744391	test: 29897.9556913	best: 29897.9556913 (600)	total: 24.4s	remaining: 1m 37s
800:	learn: 18479.8564425	test: 29336.3980533	best: 29336.3980533 (800)	total: 32.6s	remaining: 1m 29s
1000:	learn: 17554.9398769	test: 28980.4349609	best: 28978.0824800 (996)	total: 40.9s	remaining: 1m 21s
1200:	learn: 16786.5462634	test: 28674.3306293	best: 28672.7338458 (1192)	total: 49.2s	remaining: 1m 13s
1400:	learn: 16296.1075712	test: 28515.3755362	best: 28514.3109929 (1399)	total: 57.5s	remaining: 1m 5s
1600:	learn: 15839.2795740	test: 28350.0773027	best: 28350.0773027 (1600)	total: 1m 5s	remaining: 57.4s
1800:	learn: 15553.8426103	test: 28223.3956385	best: 

Predict fold 4:   0%|          | 0/2 [00:00<?, ?it/s]


=== Fold 5/5 ===
0:	learn: 80559.9233638	test: 71774.0220933	best: 71774.0220933 (0)	total: 41.8ms	remaining: 2m 5s
200:	learn: 32653.3954785	test: 29168.1725887	best: 29168.1725887 (200)	total: 8.37s	remaining: 1m 56s
400:	learn: 23449.5927260	test: 23688.1638648	best: 23688.1638648 (400)	total: 16.6s	remaining: 1m 47s
600:	learn: 20199.5496383	test: 22482.5918459	best: 22482.5419320 (599)	total: 24.6s	remaining: 1m 38s
800:	learn: 18290.4741672	test: 21856.3898960	best: 21855.3694127 (798)	total: 32.7s	remaining: 1m 29s
1000:	learn: 17029.7009715	test: 21492.7961681	best: 21492.7961681 (1000)	total: 40.9s	remaining: 1m 21s
1200:	learn: 16173.8422202	test: 21306.6673159	best: 21306.6673159 (1200)	total: 49.4s	remaining: 1m 14s
1400:	learn: 15588.6362545	test: 21228.0738388	best: 21226.6841422 (1377)	total: 57.6s	remaining: 1m 5s
1600:	learn: 15155.5664895	test: 21130.3718824	best: 21126.2622513 (1575)	total: 1m 5s	remaining: 57.5s
1800:	learn: 14818.5437622	test: 21094.7887715	best: 

Predict fold 5:   0%|          | 0/2 [00:00<?, ?it/s]


fold_rmse: [26631.27513807109, 25480.792296902735, 35933.06287096188, 27641.97976360352, 20868.98928756146]
mean_rmse: 27311.21987142014
oof_rmse: 27746.25869547771


In [17]:
import numpy as np
import pandas as pd

# предполагаем, что есть:
# - test (оригинальный тестовый df)
# - test_pred (предсказания из k-fold)
# - target_col (имя таргета, как в train)

id_col = "id"  # или "ID", или другое
for c in ["id", "ID", "Id"]:
    if c in test.columns:
        id_col = c
        break

if id_col is not None:
    sub = pd.DataFrame({
        id_col: test[id_col].values,
        target_col: test_pred
    })
else:
    sub = pd.DataFrame({
        "id": np.arange(len(test_pred)),
        target_col: test_pred
    })

sub.to_csv("submission_catboost_kfold_autofeat_gpu.csv", index=False)
sub.head()

,Id,SalePrice
0,1461,121612.924839
1,1462,160658.343975
2,1463,175427.156194
3,1464,183807.068378
4,1465,200770.568554


In [18]:
train_ag = X_tr_feat.copy()
train_ag[target_col] = y

test_ag = X_te_feat.copy()

print(train_ag.shape, test_ag.shape)
train_ag.head()

(1460, 81) (1459, 80)


,Id,MSSubClass,LotFrontage,LotArea,Street,Utilities,OverallQual,OverallCond,YearBuilt,YearRemodAdd,...,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition,SalePrice
0,1,60,65.0,8450,1,0,7,5,2003,2003,...,2,5,5,2,0,0,0,8,4,208500
1,2,20,80.0,9600,1,0,6,8,1976,1976,...,2,5,5,2,0,0,0,8,4,181500
2,3,60,68.0,11250,1,0,7,5,2001,2002,...,2,5,5,2,0,0,0,8,4,223500
3,4,70,60.0,9550,1,0,7,5,1915,1970,...,3,5,5,2,0,0,0,8,0,140000
4,5,60,84.0,14260,1,0,8,5,2000,2000,...,2,5,5,2,0,0,0,8,4,250000


In [21]:
import sys

try:
    from autogluon.tabular import TabularPredictor
    print("TabularPredictor импортирован.")
except ImportError:
    print("autogluon.tabular не найден, ставим...")
    !{sys.executable} -m pip install -q autogluon.tabular==1.4.0
    from autogluon.tabular import TabularPredictor
    print("autogluon.tabular установлен и импортирован.")

TabularPredictor импортирован.


In [22]:
predictor = TabularPredictor(
    label=target_col,
    problem_type="regression",
    eval_metric="rmse"
).fit(
    train_data=train_ag,
    time_limit=1800,  # можно 600/900/3600 по ситуации
    presets="medium_quality_faster_train",
    ag_args_fit={"num_gpus": 1},
)


No path specified. Models will be saved in: "AutogluonModels/ag-20251118_214104"
Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.10.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #40~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 16 10:53:04 UTC 2
CPU Count:          8
Memory Avail:       379.88 GB / 503.54 GB (75.4%)
Disk Space Avail:   128.69 GB / 130.00 GB (99.0%)
Presets specified: ['medium_quality_faster_train']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ... Time limit = 1800s
AutoGluon will save models to "/workspace/AutogluonModels/ag-20251118_214104"
Train Data Rows:    1460
Train Data Columns: 80
Label Column:       SalePrice
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipeli

In [38]:
print('working')

working


In [25]:
''' если нужна модель тяжелее хотим presets="best_quality" '''

'''лидерборд'''
'''
leaderboard = predictor.leaderboard(train_ag, silent=True)
leaderboard
'''

'''предсказания и сабмит'''

'''
id_col = "id"  # или "ID", или другое имя, если нужно

ag_pred = predictor.predict(test_ag)

sub_ag = test[[id_col]].copy()
sub_ag[target_col] = ag_pred

sub_ag.to_csv("submission_autogluon_autofeat_1.csv", index=False)
sub_ag.head()
'''


'\nid_col = "id"  # или "ID", или другое имя, если нужно\n\nag_pred = predictor.predict(test_ag)\n\nsub_ag = test[[id_col]].copy()\nsub_ag[target_col] = ag_pred\n\nsub_ag.to_csv("submission_autogluon_autofeat_1.csv", index=False)\nsub_ag.head()\n'

In [26]:
import sys

# 1. Проверяем, импортирован ли AutoGluon
try:
    from autogluon.tabular import TabularPredictor
except ImportError:
    print("autogluon.tabular не найден, ставим...")
    !{sys.executable} -m pip install -q autogluon.tabular==1.4.0
    from autogluon.tabular import TabularPredictor

# 2. Проверяем, существует ли predictor_cat
if "predictor_cat" not in globals():
    print("predictor_cat не найден → создаём минимальный CatBoost-предиктор…")

    # проверяем, что train_ag и target_col существуют
    assert "train_ag" in globals(), "train_ag не найден"
    assert "target_col" in globals(), "target_col не найден"

    predictor_cat = TabularPredictor(
        label=target_col,
        problem_type="regression",
        eval_metric="rmse",
    ).fit(
        train_data=train_ag,
        hyperparameters={
            "CAT": {
                "iterations": 500,   # небольшое число, чтобы не ждать
                "depth": 6,
            }
        },
        ag_args_fit={"num_gpus": 1},
        verbosity=1,
    )
    print("predictor_cat создан.")
else:
    print("predictor_cat уже существует.")

# 3. Теперь можно безопасно вызвать leaderboard
leaderboard_cat = predictor_cat.leaderboard(train_ag, silent=True)
leaderboard_cat

No path specified. Models will be saved in: "AutogluonModels/ag-20251118_214711"


predictor_cat не найден → создаём минимальный CatBoost-предиктор…


predictor_cat создан.


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,-21233.529476,-32260.18986,root_mean_squared_error,0.011924,0.013442,12.528390,0.011924,0.013442,12.528390,1,True,1
1,WeightedEnsemble_L2,-21233.529476,-32260.18986,root_mean_squared_error,0.013146,0.013669,12.529773,0.001221,0.000227,0.001383,2,True,2


In [27]:
leaderboard_cat = predictor_cat.leaderboard(train_ag, silent=True)
leaderboard_cat

ag_train_pred = predictor_cat.predict(train_ag)
ag_test_pred = predictor_cat.predict(test_ag)


In [28]:
"глюон тока с катбустом"

predictor_cat = TabularPredictor(
    label=target_col,
    problem_type="regression",
    eval_metric="rmse",
).fit(
    train_data=train_ag,
    time_limit=1800,  # можно уменьшить/увеличить
    hyperparameters={
        'CAT': {}   # включаем только CatBoost-модели
    },
    ag_args_fit={"num_gpus": 1},
)


No path specified. Models will be saved in: "AutogluonModels/ag-20251118_214749"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.10.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #40~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 16 10:53:04 UTC 2
CPU Count:          8
Memory Avail:       379.88 GB / 503.54 GB (75.4%)
Disk Space Avail:   128.63 GB / 130.00 GB (98.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory a

In [30]:
id_col = "Id"  # или "ID", или другое имя, если нужно

ag_pred = predictor.predict(test_ag)

sub_ag = test[[id_col]].copy()
sub_ag[target_col] = ag_pred

sub_ag.to_csv("submission_autogluon_autofeat_id_target.csv", index=False)
sub_ag.head()


,Id,SalePrice
0,1461,120742.289062
1,1462,160395.984375
2,1463,173950.078125
3,1464,184825.453125
4,1465,200828.468750


In [31]:
ag_pred = predictor.predict(test_ag)

sub_ag = pd.DataFrame({target_col: ag_pred})
sub_ag.to_csv("submission_autogluon_autofeat_only_id.csv", index=False)
sub_ag.head()

,SalePrice
0,120742.289062
1,160395.984375
2,173950.078125
3,184825.453125
4,200828.468750


In [32]:
from autogluon.tabular import TabularPredictor

train_ag = X_tr_feat.copy()
train_ag[target_col] = y
test_ag = X_te_feat.copy()

train_ag.shape, test_ag.shape

((1460, 81), (1459, 80))

In [33]:
predictor_cat = TabularPredictor(
    label=target_col,
    problem_type="regression",
    eval_metric="rmse",
).fit(
    train_data=train_ag,
    time_limit=1800,  # можно уменьшить/увеличить
    hyperparameters={
        'CAT': {}   # включаем только CatBoost-модели
    },
    ag_args_fit={"num_gpus": 1},
)


No path specified. Models will be saved in: "AutogluonModels/ag-20251118_215042"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.10.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #40~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 16 10:53:04 UTC 2
CPU Count:          8
Memory Avail:       379.90 GB / 503.54 GB (75.4%)
Disk Space Avail:   128.63 GB / 130.00 GB (98.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory a

In [34]:
leaderboard_cat = predictor_cat.leaderboard(train_ag, silent=True)
leaderboard_cat


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,-19612.661973,-31554.114752,root_mean_squared_error,0.01482,0.012974,36.111448,0.014820,0.012974,36.111448,1,True,1
1,WeightedEnsemble_L2,-19612.661973,-31554.114752,root_mean_squared_error,0.01607,0.013188,36.112713,0.001251,0.000214,0.001265,2,True,2


In [35]:
ag_train_pred = predictor_cat.predict(train_ag)
ag_test_pred = predictor_cat.predict(test_ag)


In [ ]:
'''blend'''

In [46]:
from catboost import CatBoostRegressor
import numpy as np

Z_train = np.vstack([
    oof,                    # OOF из твоего KFold CatBoost
    ag_train_pred.values    # предсказания AutoGluon на train
]).T

Z_test = np.vstack([
    test_pred,              # предсказания твоего KFold CatBoost на test
    ag_test_pred.values     # предсказания AutoGluon на test
]).T

meta_model = CatBoostRegressor(
    loss_function="RMSE",
    eval_metric="RMSE",
    iterations=2000,
    learning_rate=0.03,
    depth=6,
    l2_leaf_reg=3.0,
    random_seed=777,
    verbose=100,
    task_type="CPU",   # можно сменить на "GPU", devices="0"
)

meta_model.fit(Z_train, y)

blend_oof = meta_model.predict(Z_train)
blend_rmse = rmse_func(y, blend_oof)
print("blend RMSE:", blend_rmse)

blend_test = meta_model.predict(Z_test)


0:	learn: 1.1803559	total: 16.7ms	remaining: 33.3s
100:	learn: 0.9898654	total: 784ms	remaining: 14.7s
200:	learn: 0.9848862	total: 1.65s	remaining: 14.8s
300:	learn: 0.9827911	total: 2.56s	remaining: 14.4s
400:	learn: 0.9804082	total: 3.36s	remaining: 13.4s
500:	learn: 0.9781054	total: 4.17s	remaining: 12.5s
600:	learn: 0.9761623	total: 4.79s	remaining: 11.2s
700:	learn: 0.9744053	total: 5.1s	remaining: 9.45s
800:	learn: 0.9728192	total: 5.49s	remaining: 8.22s
900:	learn: 0.9713416	total: 5.91s	remaining: 7.21s
1000:	learn: 0.9699321	total: 6.33s	remaining: 6.32s
1100:	learn: 0.9686531	total: 6.74s	remaining: 5.5s
1200:	learn: 0.9674257	total: 7.16s	remaining: 4.76s
1300:	learn: 0.9662867	total: 7.59s	remaining: 4.08s
1400:	learn: 0.9651149	total: 8.02s	remaining: 3.43s
1500:	learn: 0.9640560	total: 8.43s	remaining: 2.8s
1600:	learn: 0.9631643	total: 8.83s	remaining: 2.2s
1700:	learn: 0.9622188	total: 9.25s	remaining: 1.63s
1800:	learn: 0.9613573	total: 9.67s	remaining: 1.07s
1900:	le

In [44]:
# если в test есть id-колонка
id_col = "id"  # или своё имя

sub_blend = test[[id_col]].copy()
sub_blend[target_col] = blend_test
sub_blend.to_csv("submission_blend_catboost_meta.csv", index=False)
sub_blend.head()



In [ ]:
'''далее улучшение всего вышеперечисленного (потяжелее модели) + маленько иного'''
'''почти с нуля все'''

In [42]:
DATA_DIR = Path(".")

TRAIN_PATH = DATA_DIR / "train_home.csv"   # или train.csv
TEST_PATH  = DATA_DIR / "test_home.csv"    # или test.csv
SEP = ","                             # "\t" для tsv, "," для csv

TARGET_COL = "SalePrice"            # имя таргета
ID_COL = "Id"                         # имя id-колонки в test (или None, если её нет)

train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)    # pd.read_csv(TEST_PATH,  sep=SEP)

feature_cols = [c for c in train.columns if c != TARGET_COL and c in test.columns]

X_raw = train[feature_cols].copy()
y = train[TARGET_COL].values
X_test_raw = test[feature_cols].copy()

X_raw.shape, X_test_raw.shape


((1460, 80), (1459, 80))

In [43]:
fg = AutoMLPipelineFeatureGenerator(
    enable_numeric_features=True,
    enable_categorical_features=True,
    enable_datetime_features=True,
    enable_text_special_features=True,
    enable_text_ngram_features=True,
    enable_raw_text_features=False,
)

X_tr_feat = fg.fit_transform(X=X_raw, y=y)
X_te_feat = fg.transform(X_test_raw)

X_tr_feat.shape, X_te_feat.shape


Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    388869.27 MB
	Train Data (Original)  Memory Usage: 3.85 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 3 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', [])  :  3 | ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']
		('int', [])    : 34 | ['Id', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', .

((1460, 80), (1459, 80))

In [44]:
train_ag = X_tr_feat.copy()
train_ag[TARGET_COL] = y

test_ag = X_te_feat.copy()

train_ag.shape, test_ag.shape


((1460, 81), (1459, 80))

In [45]:
'''hyperparams_base = {
    "GBM": [
        {"extra_trees": False, "ag_args": {"name_suffix": "GBM"}},
        {"extra_trees": True,  "ag_args": {"name_suffix": "GBM_XT"}},
    ],
    "CAT": {
        "iterations": 4000,
        "depth": 8,
        "learning_rate": 0.01,
    },
    "XGB": {},
    "NN_TORCH": {},
}

predictor = TabularPredictor(
    label=TARGET_COL,
    problem_type="regression",
    eval_metric="rmse",
).fit(
    train_data=train_ag,
    time_limit=1800,
    presets="medium_quality_faster_train",
    hyperparameters=hyperparams_base,
    ag_args_fit={"num_gpus": 1},
)
'''

'hyperparams_base = {\n    "GBM": [\n        {"extra_trees": False, "ag_args": {"name_suffix": "GBM"}},\n        {"extra_trees": True,  "ag_args": {"name_suffix": "GBM_XT"}},\n    ],\n    "CAT": {\n        "iterations": 4000,\n        "depth": 8,\n        "learning_rate": 0.01,\n    },\n    "XGB": {},\n    "NN_TORCH": {},\n}\n\npredictor = TabularPredictor(\n    label=TARGET_COL,\n    problem_type="regression",\n    eval_metric="rmse",\n).fit(\n    train_data=train_ag,\n    time_limit=1800,\n    presets="medium_quality_faster_train",\n    hyperparameters=hyperparams_base,\n    ag_args_fit={"num_gpus": 1},\n)\n'

In [52]:
hyperparams_extra = {
    "CAT": {
        "iterations": 4000,
        "depth": 6,
        "learning_rate": 0.05,
    },
    "GBM": [
        {"extra_trees": False, "ag_args": {"name_suffix": "GBM_extra"}},
        {"extra_trees": True,  "ag_args": {"name_suffix": "GBM_XT_extra"}},
    ],
}

predictor = predictor.fit_extra(
    hyperparameters=hyperparams_extra,
    ag_args_fit={"num_gpus": 1},
    time_limit=1200,
)


Fitting 3 L1 models, fit_strategy="sequential" ...
Fitting model: LightGBMGBM_extra ... Training model for up to 1200.00s of the 1200.00s of remaining time.
		`import lightgbm` failed. A quick tip is to install via `pip install autogluon.tabular[lightgbm]==1.4.0`.
Fitting model: LightGBMGBM_XT_extra ... Training model for up to 1199.56s of the 1199.56s of remaining time.
		`import lightgbm` failed. A quick tip is to install via `pip install autogluon.tabular[lightgbm]==1.4.0`.
Fitting model: CatBoost_2 ... Training model for up to 1199.11s of the 1199.11s of remaining time.
	Fitting with cpus=62, gpus=1, mem=2.5/427.6 GB
	Training CatBoost_2 with GPU, note that this may negatively impact model quality compared to CPU training.
	-0.9797	 = Validation score   (-root_mean_squared_error)
	30.82s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: WeightedEnsemble_2_L2 ... Training model for up to 360.00s of the 1168.20s of remaining time.
	Ensemble Weights: {'CatBoost_2': 1.0

In [53]:
lb = predictor.leaderboard(train_ag, silent=True)
lb


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,-0.999949,-0.978940,root_mean_squared_error,0.148261,0.051061,63.012222,0.148261,0.051061,63.012222,1,True,1
1,WeightedEnsemble_L2,-0.999949,-0.978940,root_mean_squared_error,0.151422,0.051380,63.014216,0.003161,0.000319,0.001994,2,True,2
2,CatBoost_2,-1.003683,-0.979668,root_mean_squared_error,0.102016,0.051478,30.824694,0.102016,0.051478,30.824694,1,True,3
3,WeightedEnsemble_2_L2,-1.003683,-0.979668,root_mean_squared_error,0.104718,0.051771,30.826800,0.002702,0.000293,0.002107,2,True,4


In [54]:
fi = predictor.feature_importance(train_ag)
fi.head(20)


Computing feature importance via permutation shuffling for 916 features using 5000 rows with 5 shuffle sets...
	1571.19s	= Expected runtime (314.24s per shuffle set)
	133.35s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
category,0.190031,0.003282,1.067495e-08,5,0.196789,0.183273
name,0.095356,0.005808,1.643011e-06,5,0.107314,0.083398
id,0.011836,0.002606,2.648099e-04,5,0.017202,0.006469
coordinates,0.008139,0.001545,1.486646e-04,5,0.011320,0.004957
address,0.005589,0.001491,5.540305e-04,5,0.008659,0.002519
__nlp__.москва,0.000860,0.000214,4.270179e-04,5,0.001301,0.000418
mean_income_1000m,0.000773,0.000084,1.655252e-05,5,0.000946,0.000600
__nlp__.троицк,0.000737,0.000363,5.234521e-03,5,0.001484,-0.000010
__nlp__.зеленоград,0.000690,0.000302,3.444998e-03,5,0.001311,0.000069
mean_income_300m,0.000645,0.000161,4.294521e-04,5,0.000977,0.000314


In [55]:
important_features = fi[fi["importance"] > 0].index.tolist()
len(important_features)


726

In [56]:
train_ag_imp = train_ag[important_features + [TARGET_COL]].copy()
test_ag_imp  = test_ag[important_features].copy()

train_ag_imp.shape, test_ag_imp.shape


((41105, 727), (9276, 726))

In [57]:
hyperparams_slim = {
    "CAT": {
        "iterations": 4000,
        "depth": 7,
        "learning_rate": 0.01,
    },
    "GBM": [
        {"extra_trees": False, "ag_args": {"name_suffix": "GBM_slim"}},
    ],
}

predictor_slim = TabularPredictor(
    label=TARGET_COL,
    problem_type="regression",
    eval_metric="mse",
).fit(
    train_data=train_ag_imp,
    time_limit=1200,
    presets="medium_quality_faster_train",
    hyperparameters=hyperparams_slim,
    ag_args_fit={"num_gpus": 1},
)

No path specified. Models will be saved in: "AutogluonModels/ag-20251116_220420"
Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #52~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Mon Dec  9 15:00:52 UTC 2
CPU Count:          62
Memory Avail:       423.48 GB / 503.46 GB (84.1%)
Disk Space Avail:   126.92 GB / 130.00 GB (97.6%)
Presets specified: ['medium_quality_faster_train']
Beginning AutoGluon training ... Time limit = 1200s
AutoGluon will save models to "/workspace/121/AutogluonModels/ag-20251116_220420"
Train Data Rows:    41105
Train Data Columns: 726
Label Column:       target
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:            

In [58]:
test_pred_base = predictor.predict(test_ag)
test_pred_slim = predictor_slim.predict(test_ag_imp)

test_pred_blend = 0.5 * test_pred_base.values + 0.5 * test_pred_slim.values


In [59]:
if ID_COL is not None and ID_COL in test.columns:
    sub_base = test[[ID_COL]].copy()
    sub_slim = test[[ID_COL]].copy()
    sub_blend = test[[ID_COL]].copy()
else:
    sub_base = pd.DataFrame()
    sub_slim = pd.DataFrame()
    sub_blend = pd.DataFrame()

sub_base[TARGET_COL] = test_pred_base
sub_slim[TARGET_COL] = test_pred_slim
sub_blend[TARGET_COL] = test_pred_blend

sub_base.to_csv("submission_ag_base.csv", index=False)
sub_slim.to_csv("submission_ag_slim.csv", index=False)
sub_blend.to_csv("submission_ag_blend.csv", index=False)

sub_blend.head()


,id,target
0,21472,3.402225
1,9837,3.115150
2,41791,3.925786
3,18441,3.096871
4,49348,2.726351


In [ ]:
'''сейчас попробую взять еще толще модели, а то скор сильно не меняется'''

In [60]:
heavy_hps = {
    "GBM": [
        {
            "extra_trees": False,
            "ag_args": {"name_suffix": "GBM_main"},
            "num_leaves": 64,
            "feature_fraction": 0.9,
            "bagging_fraction": 0.9,
            "bagging_freq": 1,
        },
        {
            "extra_trees": True,
            "ag_args": {"name_suffix": "GBM_xt"},
            "num_leaves": 128,
            "feature_fraction": 0.8,
            "bagging_fraction": 0.8,
            "bagging_freq": 1,
        },
    ],
    "CAT": [
        {
            "iterations": 6000,
            "depth": 8,
            "learning_rate": 0.03,
            "l2_leaf_reg": 3.0,
            "border_count": 254,
            "random_strength": 0.5,
            "bagging_temperature": 0.8,
            "ag_args": {"name_suffix": "CAT_main"},
        },
        {
            "iterations": 8000,
            "depth": 10,
            "learning_rate": 0.02,
            "l2_leaf_reg": 4.0,
            "border_count": 254,
            "random_strength": 0.3,
            "bagging_temperature": 0.5,
            "ag_args": {"name_suffix": "CAT_deep"},
        },
    ],
    "XGB": {
        "max_depth": 8,
        "subsample": 0.9,
        "colsample_bytree": 0.9,
        "n_estimators": 4000,
        "learning_rate": 0.03,
    },
    "NN_TORCH": {
        "ag_args": {"name_suffix": "NN_heavy"},
        "layers": "128-64-64",
        "dropout_prob": 0.1,
    },
}


In [61]:
from autogluon.tabular import TabularPredictor

heavy_time_limit = 7200  # 2 часа, под задачу можно меньше/больше

predictor_heavy = TabularPredictor(
    label=TARGET_COL,
    problem_type="regression",
    eval_metric="mse",
).fit(
    train_data=train_ag,
    time_limit=heavy_time_limit,
    presets="best_quality",          # heavy режим
    num_bag_folds=5,                 # бэггинг по фолдам
    num_bag_sets=2,                  # повторение бэггинга
    num_stack_levels=2,              # двухуровневый stacking
    hyperparameters=heavy_hps,
    ag_args_fit={"num_gpus": 1},
)


No path specified. Models will be saved in: "AutogluonModels/ag-20251116_221138"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #52~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Mon Dec  9 15:00:52 UTC 2
CPU Count:          62
Memory Avail:       427.52 GB / 503.46 GB (84.9%)
Disk Space Avail:   126.78 GB / 130.00 GB (97.5%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=2, num_bag_folds=5, num_bag_sets=2
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of 

In [62]:
extra_hps_heavy = {
    "CAT": {
        "iterations": 8000,
        "depth": 8,
        "learning_rate": 0.008,
        "l2_leaf_reg": 3.0,
        "ag_args": {"name_suffix": "CAT_extra"},
    },
    "GBM": [
        {
            "extra_trees": False,
            "ag_args": {"name_suffix": "GBM_extra"},
            "num_leaves": 128,
            "feature_fraction": 0.85,
            "bagging_fraction": 0.85,
            "bagging_freq": 1,
        },
    ],
}

predictor_heavy = predictor_heavy.fit_extra(
    hyperparameters=extra_hps_heavy,
    ag_args_fit={"num_gpus": 1},
    time_limit=1800,
)


Fitting 2 L1 models, fit_strategy="sequential" ...
Fitting model: LightGBMGBM_extra_BAG_L1 ... Training model for up to 1800.00s of the 1800.00s of remaining time.
		`import lightgbm` failed. A quick tip is to install via `pip install autogluon.tabular[lightgbm]==1.4.0`.
Fitting model: CatBoostCAT_extra_BAG_L1 ... Training model for up to 1799.58s of the 1799.58s of remaining time.
	Fitting 10 child models (S1F1 - S2F5) | Fitting with SequentialLocalFoldFittingStrategy (sequential: cpus=62, gpus=1)
	Training S1F1 with GPU, note that this may negatively impact model quality compared to CPU training.
	Training S1F2 with GPU, note that this may negatively impact model quality compared to CPU training.
	Training S1F3 with GPU, note that this may negatively impact model quality compared to CPU training.
	Training S1F4 with GPU, note that this may negatively impact model quality compared to CPU training.
	Training S1F5 with GPU, note that this may negatively impact model quality compared to 

In [63]:
refit_models = predictor_heavy.refit_full()  # вернёт список имён моделей
refit_models


Refitting models via `predictor.refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix "_FULL" and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `predictor.fit` call.
	To learn more, refer to the `.refit_full` method docstring which explains how "_FULL" models differ from normal models.
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: CatBoostCAT_main_BAG_L1_FULL ...
	Training S1F1 with GPU, note that this may negatively impact model quality compared to CPU training.
	47.04s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: CatBoostCAT_deep_BAG_L1_FULL ...
	Training S1F1 with GPU, note that this may negatively impact model quality compared to CPU training.
	155.07s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: CatBoostCAT_extra_BAG_L1_FULL ...
	Training S1F1 with

{'CatBoostCAT_main_BAG_L1': 'CatBoostCAT_main_BAG_L1_FULL',
 'CatBoostCAT_deep_BAG_L1': 'CatBoostCAT_deep_BAG_L1_FULL',
 'CatBoostCAT_extra_BAG_L1': 'CatBoostCAT_extra_BAG_L1_FULL',
 'WeightedEnsemble_L2': 'WeightedEnsemble_L2_FULL',
 'CatBoostCAT_main_BAG_L2': 'CatBoostCAT_main_BAG_L2_FULL',
 'CatBoostCAT_deep_BAG_L2': 'CatBoostCAT_deep_BAG_L2_FULL',
 'WeightedEnsemble_2_L2': 'WeightedEnsemble_2_L2_FULL',
 'WeightedEnsemble_L3': 'WeightedEnsemble_L3_FULL',
 'CatBoostCAT_main_BAG_L3': 'CatBoostCAT_main_BAG_L3_FULL',
 'CatBoostCAT_deep_BAG_L3': 'CatBoostCAT_deep_BAG_L3_FULL',
 'WeightedEnsemble_L4': 'WeightedEnsemble_L4_FULL'}

In [64]:
lb_heavy = predictor_heavy.leaderboard(train_ag, silent=True)
lb_heavy


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoostCAT_deep_BAG_L3_FULL,-0.916279,NaN,root_mean_squared_error,0.599813,NaN,225.043656,0.099124,NaN,8.463934,3,True,21
1,CatBoostCAT_main_BAG_L3_FULL,-0.923221,NaN,root_mean_squared_error,0.584995,NaN,221.072683,0.084306,NaN,4.492962,3,True,20
2,CatBoostCAT_deep_BAG_L2_FULL,-0.923662,NaN,root_mean_squared_error,0.411435,NaN,211.649469,0.100992,NaN,9.546101,2,True,17
3,WeightedEnsemble_L3_FULL,-0.924691,NaN,root_mean_squared_error,0.501887,NaN,216.589046,0.001198,NaN,0.009325,3,True,19
4,WeightedEnsemble_L4_FULL,-0.925978,NaN,root_mean_squared_error,0.413476,NaN,211.667761,0.002040,NaN,0.018292,4,True,22
5,CatBoostCAT_main_BAG_L2_FULL,-0.935022,NaN,root_mean_squared_error,0.399697,NaN,207.033620,0.089254,NaN,4.930252,2,True,16
6,CatBoostCAT_deep_BAG_L3,-0.943104,-1.020547,root_mean_squared_error,2.916770,3.570393,3347.472876,0.467275,0.723779,460.999281,3,True,8
7,CatBoostCAT_main_BAG_L3,-0.944375,-1.020356,root_mean_squared_error,2.881808,3.521692,3041.426079,0.432313,0.675078,154.952484,3,True,7
8,CatBoostCAT_deep_BAG_L2,-0.946427,-1.019281,root_mean_squared_error,2.014383,2.123481,2710.522289,0.443499,0.746373,518.578130,2,True,5
9,WeightedEnsemble_L3,-0.946615,-1.019280,root_mean_squared_error,2.451519,2.847040,2886.482920,0.002024,0.000426,0.009325,3,True,6


In [65]:
fi_heavy = predictor_heavy.feature_importance(train_ag)
fi_heavy.head(30)


Computing feature importance via permutation shuffling for 916 features using 5000 rows with 5 shuffle sets...
	1222.19s	= Expected runtime (244.44s per shuffle set)
	160.98s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
category,0.256087,0.006250,4.254124e-08,5,0.268956,0.243218
name,0.143222,0.009931,2.756245e-06,5,0.163670,0.122774
id,0.026039,0.003964,6.249724e-05,5,0.034201,0.017877
coordinates,0.009665,0.001618,9.073447e-05,5,0.012996,0.006335
address,0.007237,0.001181,8.226482e-05,5,0.009669,0.004805
mean_income_1000m,0.004080,0.000457,1.859026e-05,5,0.005022,0.003139
mean_income_300m,0.004065,0.000653,7.712692e-05,5,0.005409,0.002721
__nlp__.москва,0.003000,0.000750,4.327484e-04,5,0.004545,0.001455
address.symbol_count..,0.002978,0.000774,5.027841e-04,5,0.004572,0.001383
homes_300m,0.002896,0.000545,1.432752e-04,5,0.004017,0.001775


In [66]:
test_pred_heavy = predictor_heavy.predict(test_ag)
test_pred_heavy.head()


0    3.433025
1    3.425081
2    4.002472
3    3.274601
4    2.883332
Name: target, dtype: float32

In [67]:
# Предсказания medium (если predictor ещё есть)
try:
    test_pred_medium = predictor.predict(test_ag)
except NameError:
    test_pred_medium = None

if ID_COL is not None and ID_COL in test.columns:
    sub_heavy = test[[ID_COL]].copy()
    sub_blend = test[[ID_COL]].copy()
else:
    sub_heavy = pd.DataFrame()
    sub_blend = pd.DataFrame()

sub_heavy[TARGET_COL] = test_pred_heavy

if test_pred_medium is not None:
    test_pred_blend = 0.5 * test_pred_heavy.values + 0.5 * test_pred_medium.values
    sub_blend[TARGET_COL] = test_pred_blend
else:
    sub_blend[TARGET_COL] = test_pred_heavy.values

sub_heavy.to_csv("submission_ag_heavy.csv", index=False)
sub_blend.to_csv("submission_ag_heavy_blend.csv", index=False)

sub_heavy.head()


,id,target
0,21472,3.433025
1,9837,3.425081
2,41791,4.002472
3,18441,3.274601
4,49348,2.883332
